# ES → BBF Salesforce Opportunity Migration

This notebook migrates Opportunity records from ES Salesforce to BBF Salesforce.

## Prerequisites
- **Account migration must be completed first (Wave 1)**
- ES Accounts with `BBF_New_Id__c` populated = parent Account already exists in BBF
- ES Opportunities marked with `BBF_Opportunity__c = True` = records to migrate

## Object Mapping
- **ES Source:** Opportunity (357 fields)
- **BBF Target:** Opportunity (284 fields)

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query Opportunities from ES where:
   - `BBF_Opportunity__c = True` (marked for migration)
   - Parent Account has `BBF_New_Id__c` populated (Account already migrated)
   - `BBF_New_Id__c` is empty (not yet migrated)
3. Transform ES Opportunity for BBF schema:
   - Map `AccountId` → ES Account's `BBF_New_Id__c` (BBF Account ID)
   - Map StageName picklist values (ES → BBF)
   - Set 26+ required boolean fields to FALSE
   - Add `ES_Legacy_ID__c` = ES Opportunity.Id (for tracking)
4. Insert Opportunities to BBF Salesforce
5. Update ES Opportunities with `BBF_New_Id__c` = BBF Opportunity.Id
6. Create ID mapping: ES Opportunity ID → BBF Opportunity ID
7. Output results to Excel with color-coded status

## Stage Mapping (ES → BBF)
| ES Stage | BBF Stage |
|----------|----------|
| Opportunity Identified | Opportunity |
| In Progress | Scoping |
| Quote Presented | Proposed |
| Waiting For Docusign | Contracting |
| Sold: SOF Requires Activation | Customer Executed |
| Sold: SOF Activated | Closed Won Complete |
| Closed Lost | Closed Lost |

## Field Tracking Strategy
**In BBF Opportunity:** `ES_Legacy_ID__c` stores original ES Opportunity ID
- Text(18), External ID, Unique

**In ES Opportunity:**
- `BBF_Opportunity__c` = True (checkbox) - **pre-set to identify records to migrate**
- `BBF_New_Id__c` stores new BBF Opportunity ID after migration (Text 18)

## Safety
- `TEST_MODE = True` by default (limits to 10 Opportunities)
- Only migrates Opportunities where `BBF_Opportunity__c = True`
- Skips Opportunities where parent Account not yet migrated
- Skips Opportunities already migrated (`BBF_New_Id__c` populated)

In [ ]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

In [ ]:
# === CONFIGURATION ===

# ES (Source) Credentials
ES_USERNAME = "vlettau@everstream.net.uat"
ES_PASSWORD = "MNlkpo0987)(*&"
ES_TOKEN = "nSBoNS97wYLCRW2JP2JARR12"
ES_DOMAIN = "test"  # or 'login' for production

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = True  # ⚠️ Set to False to migrate ALL Opportunities
TEST_LIMIT = 10   # Only used when TEST_MODE = True

# 👤 Opportunity Owner - Set all migrated Opportunities to this user
OWNER_ID = "005Ea00000ZOGFZIA5"  # Same as Account/Contact/BAN migration

# 🏢 Default Business Unit - For EVS opportunities
DEFAULT_BUS_UNIT = "EVS"  # EverStream business unit in BBF

# 📋 Stage Mapping: ES Stage → BBF Stage
STAGE_MAPPING = {
    "Opportunity Identified": "Opportunity",
    "In Progress": "Scoping",
    "Quote Presented": "Proposed",
    "Waiting For Docusign": "Contracting",
    "Sold: SOF Requires Activation": "Customer Executed",
    "Sold: SOF Activated": "Closed Won Complete",
    "Closed Lost": "Closed Lost",
}

# Valid BBF Stages (for validation)
VALID_BBF_STAGES = [
    "Contacting", "Opportunity", "Scoping", "Sales Engineering", "OMG Review",
    "Proposed", "Verbal", "Sales Engineering Final Review", "Sales Leader Review",
    "Contract Management", "Contracting", "Closed Won", "Customer Executed",
    "Countersignature", "Closed Won Complete", "Closed Lost", "Discard",
    "Cancelled", "Rejected"
]

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_opportunity_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Stage Mappings: {len(STAGE_MAPPING)}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

In [ ]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n🔌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n🔌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

In [ ]:
# === QUERY ES OPPORTUNITIES ===

print("\n" + "=" * 80)
print("QUERYING ES OPPORTUNITIES")
print("=" * 80)

# Build query - get Opportunities marked for migration where Account is already migrated
limit_clause = f"LIMIT {TEST_LIMIT}" if TEST_MODE else ""

query = f"""
    SELECT Id, Name, AccountId, Account.Name, Account.BBF_New_Id__c,
           StageName, CloseDate, Amount, Probability, Type, LeadSource,
           Description, NextStep, IsClosed, IsWon,
           
           /* Financial Fields */
           Total_MRC__c, Total_NRC_Required__c, Sold_MRC__c, Sold_NRC__c,
           Pipeline_Value_MRC__c, CPQ_MRC__c, CPQ_NRC__c,
           Cost_Equipment__c, Cost_Construction__c, Cost_Total_1C_Cost2__c,
           
           /* Location A Fields */
           LOC_A_Address__c, LOC_A_City__c, LOC_A_State__c, LOC_A_Zip2__c,
           A_Loc_Site_Name__c, LOC_A_County__c,
           
           /* Location Z Fields */
           LOC_Z_Address__c, LOC_Z_City__c, LOC_Z_State__c, LOC_Z_Zip2__c,
           LOC_Z_Address_2__c, LOC_Z_County__c,
           
           /* Service/Product Fields */
           Service_Category__c, Service_Term__c, Product_Term_Roll_Up__c,
           Diversity_Type__c, Diversity_Entrance__c, Diversity_Fiber_Path__c,
           
           /* Contact References */
           Lead_Contact_Primary__c, Lead_Contact_Secondary__c,
           
           /* Status/Workflow Fields */
           Opportunity_ID_Number__c, Customer_Requested_Due_Date__c,
           Deal_Lost_Reason__c, Special_Conditions_Requests__c,
           Salesperson_Latest_Update__c,
           
           /* Dates */
           CreatedDate, LastModifiedDate,
           
           /* Tracking */
           BBF_New_Id__c
           
    FROM Opportunity
    WHERE BBF_Opportunity__c = true
      AND Account.BBF_New_Id__c != null
      AND BBF_New_Id__c = null
    ORDER BY CreatedDate DESC
    {limit_clause}
"""

print(f"Query: {query[:500]}...")
print("\n🔍 Executing query...")

try:
    es_opps_result = es_sf.query_all(query)
    es_opps = es_opps_result['records']
    print(f"✅ Found {len(es_opps)} Opportunities to migrate")
    
    if len(es_opps) > 0:
        # Show sample
        print("\n📊 Sample Opportunities:")
        for i, opp in enumerate(es_opps[:5]):
            acct_name = opp.get('Account', {}).get('Name', 'N/A') if opp.get('Account') else 'N/A'
            print(f"   {i+1}. {opp['Name'][:50]} | Stage: {opp['StageName']} | Account: {acct_name[:30]}")
        
        # Stage distribution
        stages = {}
        for opp in es_opps:
            stage = opp['StageName']
            stages[stage] = stages.get(stage, 0) + 1
        print("\n📈 Stage Distribution:")
        for stage, count in sorted(stages.items(), key=lambda x: -x[1]):
            bbf_stage = STAGE_MAPPING.get(stage, "⚠️ UNMAPPED")
            print(f"   {stage}: {count} → BBF: {bbf_stage}")
            
except Exception as e:
    print(f"❌ Query failed: {e}")
    print("\n💡 If BBF_Opportunity__c field doesn't exist, you may need to:")
    print("   1. Create the field in ES Salesforce, OR")
    print("   2. Modify the query to use a different filter criteria")
    es_opps = []

In [ ]:
# === PRE-VALIDATE STAGE MAPPINGS ===

print("\n" + "=" * 80)
print("PRE-VALIDATING STAGE MAPPINGS")
print("=" * 80)

# Check for unmapped stages
unmapped_stages = []
stage_counts = {}

for opp in es_opps:
    es_stage = opp['StageName']
    stage_counts[es_stage] = stage_counts.get(es_stage, 0) + 1
    
    if es_stage not in STAGE_MAPPING:
        if es_stage not in unmapped_stages:
            unmapped_stages.append(es_stage)

if unmapped_stages:
    print("\n⚠️  WARNING: Found unmapped ES stages:")
    for stage in unmapped_stages:
        count = stage_counts.get(stage, 0)
        print(f"   - '{stage}' ({count} records)")
    print("\n   These will be mapped to 'Opportunity' (default)")
    print("   Update STAGE_MAPPING in Cell 2 if different mapping needed.")
else:
    print("\n✅ All ES stages have valid BBF mappings")

# Validate BBF target stages exist
print("\n🔍 Validating target BBF stages...")
invalid_targets = []
for es_stage, bbf_stage in STAGE_MAPPING.items():
    if bbf_stage not in VALID_BBF_STAGES:
        invalid_targets.append((es_stage, bbf_stage))

if invalid_targets:
    print("\n❌ ERROR: Invalid BBF target stages in mapping:")
    for es_stage, bbf_stage in invalid_targets:
        print(f"   '{es_stage}' → '{bbf_stage}' (NOT VALID)")
    print("\n   Fix STAGE_MAPPING before proceeding!")
else:
    print("✅ All BBF target stages are valid")

In [ ]:
# === TRANSFORM ES OPPORTUNITIES FOR BBF ===

print("\n" + "=" * 80)
print("TRANSFORMING ES OPPORTUNITIES FOR BBF")
print("=" * 80)

bbf_opps = []
skipped_no_account = []
skipped_unmapped_stage = []
transform_errors = []

for opp in es_opps:
    try:
        # Get BBF Account ID from parent Account
        bbf_account_id = None
        if opp.get('Account') and opp['Account'].get('BBF_New_Id__c'):
            bbf_account_id = opp['Account']['BBF_New_Id__c']
        
        if not bbf_account_id:
            skipped_no_account.append({
                "es_id": opp['Id'],
                "name": opp['Name'],
                "reason": "Parent Account not migrated (BBF_New_Id__c is null)"
            })
            continue
        
        # Map Stage
        es_stage = opp['StageName']
        bbf_stage = STAGE_MAPPING.get(es_stage, "Opportunity")  # Default to 'Opportunity'
        
        # Build BBF Opportunity record
        bbf_opp = {
            # === REQUIRED FIELDS ===
            "Name": opp['Name'][:120] if opp.get('Name') else "Migrated Opportunity",
            "AccountId": bbf_account_id,
            "StageName": bbf_stage,
            "CloseDate": opp['CloseDate'],
            "OwnerId": OWNER_ID,
            
            # Project__c and Opportunity_Number__c are required strings
            # Use ES values if available, otherwise generate
            "Project__c": opp.get('Opportunity_ID_Number__c') or f"ES-{opp['Id'][-8:]}",
            "Opportunity_Number__c": opp.get('Opportunity_ID_Number__c') or f"ES-{opp['Id'][-10:]}",
            
            # === REQUIRED BOOLEANS (all default FALSE) ===
            "Budget_Confirmed__c": False,
            "Discovery_Completed__c": False,
            "ROI_Analysis_Completed__c": False,
            "Override_Closed_Date__c": False,
            "Email_OSP__c": False,
            "Email_Sales_Engineering__c": False,
            "Agent__c": False,
            "Contract_Management_Approval_Required__c": False,
            "OSP_Engineering_Approval_Required__c": False,
            "ISP_Engineering_Approval_Required__c": False,
            "Service_Delivery_Approval_Required__c": False,
            "Operations_Approval_Required__c": False,
            "Contract_Management_Approval_Complete__c": False,
            "OSP_Engineering_Approval_Complete__c": False,
            "ISP_Engineering_Approval_Complete__c": False,
            "Service_Delivery_Approval_Complete__c": False,
            "Operations_Approval_Complete__c": False,
            "Is_Near_Net_Opportunity__c": False,
            "Approved__c": False,
            "Override_Automated_Calculation__c": False,
            "Lock_Opportunity__c": False,
            "Re_Trigger_Flow__c": False,
            "OSP_Required__c": False,
            "USAC__c": False,
            "Prevailing_Wages__c": False,
            "Is_there_a_BAN__c": False,
            "Is_Wireless_Order__c": False,
            "Off_Net_Review_Complete__c": False,
            "SE_Manager_Review_Complete__c": False,
            "TASKRAY__trOnboardingKickoffCompleted__c": False,
            "Split_Commissions_Manually__c": False,
            "Has_Dark_Fiber__c": False,
            "Override_NPV__c": False,
            
            # === OPTIONAL STANDARD FIELDS ===
            "Amount": opp.get('Amount'),
            "Probability": opp.get('Probability'),
            "Type": opp.get('Type'),  # May need picklist mapping
            "LeadSource": opp.get('LeadSource'),  # May need picklist mapping
            "Description": opp.get('Description'),
            "NextStep": opp.get('NextStep')[:255] if opp.get('NextStep') else None,
            
            # === FINANCIAL FIELDS (map ES → BBF) ===
            # Note: Field names may differ - adjust as needed
            # "Total_MRC__c": opp.get('Total_MRC__c'),
            # "Total_NRC__c": opp.get('Total_NRC_Required__c'),
            
            # === TRACKING ===
            "ES_Legacy_ID__c": opp['Id'],
        }
        
        # Store ES data for reference (will go in Excel, not SF)
        bbf_opp['_es_stage'] = es_stage
        bbf_opp['_es_account_name'] = opp.get('Account', {}).get('Name', '') if opp.get('Account') else ''
        bbf_opp['_es_total_mrc'] = opp.get('Total_MRC__c')
        bbf_opp['_es_close_date'] = opp.get('CloseDate')
        
        # Remove None values (Salesforce doesn't like them)
        bbf_opp_clean = {k: v for k, v in bbf_opp.items() if v is not None and not k.startswith('_')}
        bbf_opp_clean['_metadata'] = {k: v for k, v in bbf_opp.items() if k.startswith('_')}
        
        bbf_opps.append(bbf_opp_clean)
        
    except Exception as e:
        transform_errors.append({
            "es_id": opp.get('Id', 'Unknown'),
            "name": opp.get('Name', 'Unknown'),
            "error": str(e)
        })

print(f"\n📊 Transformation Results:")
print(f"   ✅ Ready to migrate: {len(bbf_opps)}")
print(f"   ⚠️  Skipped (no Account mapping): {len(skipped_no_account)}")
print(f"   ❌ Transform errors: {len(transform_errors)}")

if skipped_no_account:
    print("\n⚠️  Skipped Opportunities (Account not migrated):")
    for item in skipped_no_account[:5]:
        print(f"   - {item['name'][:50]}")
    if len(skipped_no_account) > 5:
        print(f"   ... and {len(skipped_no_account) - 5} more")

if transform_errors:
    print("\n❌ Transform Errors:")
    for item in transform_errors[:5]:
        print(f"   - {item['name'][:40]}: {item['error']}")

In [ ]:
# === DRY RUN - PREVIEW BEFORE INSERT ===

print("\n" + "=" * 80)
print("DRY RUN - PREVIEW OPPORTUNITIES TO INSERT")
print("=" * 80)

if len(bbf_opps) == 0:
    print("\n⚠️  No Opportunities to migrate. Check previous cells for errors.")
else:
    print(f"\n📋 Will insert {len(bbf_opps)} Opportunities to BBF")
    print("\n🔍 Sample records (first 5):")
    print("-" * 100)
    
    for i, opp in enumerate(bbf_opps[:5]):
        metadata = opp.get('_metadata', {})
        print(f"\n{i+1}. {opp['Name'][:60]}")
        print(f"   ES Stage: {metadata.get('_es_stage', 'N/A')} → BBF Stage: {opp['StageName']}")
        print(f"   Account: {metadata.get('_es_account_name', 'N/A')[:40]} → BBF ID: {opp['AccountId']}")
        print(f"   Close Date: {opp['CloseDate']}")
        print(f"   ES Legacy ID: {opp['ES_Legacy_ID__c']}")
    
    print("\n" + "-" * 100)
    print(f"\n⚠️  Ready to insert {len(bbf_opps)} records.")
    print("   Run the next cell to execute the insert.")

In [ ]:
# === INSERT OPPORTUNITIES TO BBF ===

print("\n" + "=" * 80)
print("INSERTING OPPORTUNITIES TO BBF")
print("=" * 80)

successful_inserts = []
failed_inserts = []

if len(bbf_opps) == 0:
    print("\n⚠️  No Opportunities to insert.")
else:
    # Prepare records for bulk insert (remove metadata)
    insert_records = []
    metadata_map = {}  # ES ID → metadata
    
    for opp in bbf_opps:
        es_id = opp['ES_Legacy_ID__c']
        metadata_map[es_id] = opp.pop('_metadata', {})
        insert_records.append(opp)
    
    print(f"\n🚀 Inserting {len(insert_records)} Opportunities via Bulk API...")
    
    try:
        # Use Bulk API for efficient insert
        results = bbf_sf.bulk.Opportunity.insert(insert_records, batch_size=200)
        
        # Process results
        for i, result in enumerate(results):
            record = insert_records[i]
            es_id = record['ES_Legacy_ID__c']
            metadata = metadata_map.get(es_id, {})
            
            if result['success']:
                successful_inserts.append({
                    "es_id": es_id,
                    "bbf_id": result['id'],
                    "name": record['Name'],
                    "bbf_account_id": record['AccountId'],
                    "es_stage": metadata.get('_es_stage', ''),
                    "bbf_stage": record['StageName'],
                    "close_date": record['CloseDate'],
                })
            else:
                failed_inserts.append({
                    "es_id": es_id,
                    "name": record['Name'],
                    "bbf_account_id": record['AccountId'],
                    "errors": result.get('errors', 'Unknown error'),
                })
        
        print(f"\n✅ Successfully inserted: {len(successful_inserts)}")
        print(f"❌ Failed: {len(failed_inserts)}")
        
        if failed_inserts:
            print("\n❌ Failed Inserts:")
            for item in failed_inserts[:10]:
                print(f"   - {item['name'][:40]}: {item['errors']}")
            if len(failed_inserts) > 10:
                print(f"   ... and {len(failed_inserts) - 10} more")
                
    except Exception as e:
        print(f"\n❌ Bulk insert failed: {e}")
        print("\n💡 Try inserting in smaller batches or check field mappings.")

In [ ]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES OPPORTUNITIES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("\n⚠️  No successful inserts to update.")
else:
    # Prepare update records
    update_records = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]
    
    print(f"\n🔄 Updating {len(update_records)} ES Opportunities with BBF IDs...")
    
    try:
        update_results = es_sf.bulk.Opportunity.update(update_records, batch_size=200)
        
        update_success = sum(1 for r in update_results if r['success'])
        update_failed = sum(1 for r in update_results if not r['success'])
        
        print(f"\n✅ ES records updated: {update_success}")
        if update_failed > 0:
            print(f"❌ ES update failures: {update_failed}")
            for i, r in enumerate(update_results):
                if not r['success']:
                    print(f"   - {update_records[i]['Id']}: {r.get('errors', 'Unknown')}")
                    if i >= 5:
                        print(f"   ... and more")
                        break
    except Exception as e:
        print(f"\n❌ ES update failed: {e}")
        print("\n💡 BBF records were created successfully.")
        print("   ES records need manual update of BBF_New_Id__c field.")

In [ ]:
# === GENERATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("GENERATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
success_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
error_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
skip_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")

# --- SHEET 1: Migration Results ---
ws1 = wb.active
ws1.title = "Migration Results"

headers1 = [
    "Status", "ES Opportunity ID", "BBF Opportunity ID", "Opportunity Name",
    "ES Stage", "BBF Stage", "Close Date", "BBF Account ID", "Error/Notes"
]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

# Add successful inserts
for item in successful_inserts:
    row = [
        "SUCCESS", item["es_id"], item["bbf_id"], item["name"],
        item["es_stage"], item["bbf_stage"], item["close_date"],
        item["bbf_account_id"], ""
    ]
    ws1.append(row)
    for col in range(1, len(row) + 1):
        ws1.cell(row=ws1.max_row, column=col).fill = success_fill

# Add failed inserts
for item in failed_inserts:
    row = [
        "FAILED", item["es_id"], "", item["name"],
        "", "", "", item["bbf_account_id"], str(item["errors"])
    ]
    ws1.append(row)
    for col in range(1, len(row) + 1):
        ws1.cell(row=ws1.max_row, column=col).fill = error_fill

# Add skipped records
for item in skipped_no_account:
    row = [
        "SKIPPED", item["es_id"], "", item["name"],
        "", "", "", "", item["reason"]
    ]
    ws1.append(row)
    for col in range(1, len(row) + 1):
        ws1.cell(row=ws1.max_row, column=col).fill = skip_fill

# Auto-width columns
for col in ws1.columns:
    max_length = max(len(str(cell.value or "")) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
summary_data = [
    ["ES → BBF Opportunity Migration Summary", ""],
    ["", ""],
    ["Migration Date", datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
    ["Test Mode", "Yes" if TEST_MODE else "No"],
    ["", ""],
    ["ES Opportunities Queried", len(es_opps)],
    ["Successfully Migrated", len(successful_inserts)],
    ["Failed to Insert", len(failed_inserts)],
    ["Skipped (No Account Mapping)", len(skipped_no_account)],
    ["Transform Errors", len(transform_errors)],
    ["", ""],
    ["Success Rate", f"{len(successful_inserts)/max(len(es_opps),1)*100:.1f}%"],
    ["", ""],
    ["Stage Mapping Used:", ""],
]

for es_stage, bbf_stage in STAGE_MAPPING.items():
    summary_data.append([f"  {es_stage}", f"→ {bbf_stage}"])

for row in summary_data:
    ws2.append(row)

ws2.column_dimensions['A'].width = 40
ws2.column_dimensions['B'].width = 30

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES Opportunity ID", "BBF Opportunity ID", "BBF Account ID", "Opportunity Name", "BBF Stage"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append([item["es_id"], item["bbf_id"], item["bbf_account_id"], item["name"], item["bbf_stage"]])

for col in ws3.columns:
    max_length = max(len(str(cell.value or "")) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES Opportunity ID", "Opportunity Name", "BBF Account ID", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append([item["es_id"], item["name"], item["bbf_account_id"], str(item["errors"])])

for item in transform_errors:
    ws4.append([item["es_id"], item["name"], "", item["error"]])

for col in ws4.columns:
    max_length = max(len(str(cell.value or "")) for cell in col) if list(col) else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(successful_inserts) + len(failed_inserts) + len(skipped_no_account)} records, color-coded)")
print(f"   📈 Sheet 2: Summary (metrics and stage mapping)")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} successful mappings)")
print(f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts) + len(transform_errors)} failures)")

In [ ]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES Opportunities queried: {len(es_opps)}")
print(f"BBF Opportunities inserted: {len(successful_inserts)}")
print(f"Success rate: {len(successful_inserts)/max(len(es_opps),1)*100:.1f}%")
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print(f"\n🔄 TEST MODE complete. Only migrated {TEST_LIMIT} Opportunities.")
    print("   To migrate ALL Opportunities, set TEST_MODE = False in Cell 2 and re-run.")
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   Opportunity migration finished.")
    print("   Next: Migrate Opportunity_Site__c, Quote, QuoteLineItem, etc.")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} Opportunities failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")

if len(skipped_no_account) > 0:
    print(f"\n⚠️  {len(skipped_no_account)} Opportunities skipped (Account not migrated)")
    print("   These can be migrated after their parent Accounts are migrated")

---
## Next Steps: Wave 3 Objects

After Opportunity migration is complete, use the **ID Mapping sheet** to migrate related objects:

1. **Opportunity_Site__c** (needs Opportunity + Location IDs) - Junction object for multi-site opportunities
2. **Quote** (needs Opportunity + Account IDs) - Quotes linked to opportunities
3. **QuoteLineItem** (needs Quote + Product IDs) - Line items on quotes

## Field Mapping Reference

### ES Opportunity → BBF Opportunity Field Mapping

| ES Field | BBF Field | Notes |
|----------|-----------|-------|
| Id | ES_Legacy_ID__c | Tracking |
| AccountId | AccountId | Via BBF_New_Id__c lookup |
| Name | Name | Direct map (max 120 chars) |
| StageName | StageName | Picklist mapping required |
| CloseDate | CloseDate | Direct map |
| Amount | Amount | Direct map |
| Opportunity_ID_Number__c | Project__c, Opportunity_Number__c | Required strings |
| (26+ booleans) | Various | All default FALSE |

### Stage Mapping

| ES Stage | BBF Stage |
|----------|----------|
| Opportunity Identified | Opportunity |
| In Progress | Scoping |
| Quote Presented | Proposed |
| Waiting For Docusign | Contracting |
| Sold: SOF Requires Activation | Customer Executed |
| Sold: SOF Activated | Closed Won Complete |
| Closed Lost | Closed Lost |

## Cleanup Apex (if needed)

### Delete Migrated Opportunities from BBF
```apex
List<Opportunity> opps = [SELECT Id, Name FROM Opportunity WHERE ES_Legacy_ID__c != null LIMIT 200];
System.debug('Found ' + opps.size() + ' migrated Opportunities');
delete opps;
```

### Remove BBF_New_Id__c from ES (to re-run migration)
```apex
List<Opportunity> opps = [SELECT Id, BBF_New_Id__c 
                          FROM Opportunity 
                          WHERE BBF_New_Id__c != NULL LIMIT 200];
System.debug('Found ' + opps.size() + ' records to reset');
for (Opportunity opp : opps) {
    opp.BBF_New_Id__c = NULL;
}
update opps;
```

In [ ]:
# Install required packages (run if needed)
# !pip install simple-salesforce pandas openpyxl